In [1]:
"""
MODIS Leaf Area Index
"""

import ee, geemap
import os, sys, time
import pandas as pd

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Success")

Success


In [2]:
mod15 = ee.ImageCollection("MODIS/061/MOD15A2H")
mod15.limit(1)

In [75]:
# load the frp grid
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_snpp_jpss1_afd_latlon_aspenfires_pixar_gridstats')
print(f"{grid.size().getInfo()} total gridcells.")
print(grid.first().propertyNames().getInfo())

49047 total gridcells.
['Fire_Year', 'grid_index', 'Fire_ID', 'max_date', 'afd_count', 'Ig_Date', 'first_date', 'last_date', 'Last_Dat_1', 'system:index']


In [78]:
# function to extract lai/fpar from MOD15 within gridcells
def lai_fpar_stats(fire_id):
    """ calculate the LAI/FPAR for fire gridcells """
    
    grid_fire = grid.filter(ee.Filter.eq('Fire_ID',fire_id)) # get the gridcells for the current fire
    
    end_date = ee.Date(grid_fire.first().get('Ig_Date')).advance(-1,'days') # day before discovery day
    start_date = end_date.advance(-18,'days') # 16-days pre-fire

    # filter the modis collection
    mod15_ = mod15.filterDate(start_date, end_date).mean()
    
    # do the reduction
    mod15_stats = mod15_.select(['Lai_500m','Fpar_500m']).reduceRegions(
        collection=grid_fire,
        reducer=ee.Reducer.mean(),
        scale=375 # gridcell 'scale'
    ).select(['grid_index','Lai_500m','Fpar_500m'])

    return mod15_stats
    
# map the function over all fire ids
fire_ids = grid.aggregate_array('Fire_ID').distinct()
print(f"Processing: {fire_ids.size().getInfo()} fires.")

results = fire_ids.map(lai_fpar_stats)
results_fc = ee.FeatureCollection(results).flatten()

print("Number of features in LAI/FPAR stats:", results_fc.size().getInfo())

Processing: 58 fires.
Number of features in LAI/FPAR stats: 49047


In [79]:
sample = results_fc.limit(10)
sample

In [80]:
# Export the table. 
export_task = ee.batch.Export.table.toDrive(
    collection=results_fc,
    description='gridstats_mod15_lai_fpar',
    fileNamePrefix='gridstats_mod15_lai_fpar',
    fileFormat='CSV', 
    folder='LAI'
)

export_task.start() # Start the export task
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 60)

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Export completed successfully !!!!


In [ ]:
# create a training dataset for LAI over TreeMap species